Harold Articles 
Date: 23rd March 2022

In [1]:
#libaries
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
import requests
import json

In [25]:
#find class value, NZherald changes this value every 24 hours.
#return "na" value if article is free :P
def findClass(soup):    
    #<script> tag contains the css selector value
    data = json.loads(soup.find('script', type='application/ld+json').text)
    
    if 'hasPart' in data:
        #print("premiumm")
        className = data['hasPart'][0]['cssSelector'][1:]
        return className
    else:
        #print("its free silly!")
        className = "na"
        return className

In [46]:
#retrive contents of the article
def getContents(soup, value):
    #title
    if soup.find("h1",{"class":"article-bigread__heading"}):
        #if big read article
        title = soup.find("h1",{"class":"article-bigread__heading"}).text
    elif soup.find("h1",{"class":"article__heading"}):
        #normal article
        title = soup.find("h1",{"class":"article__heading"}).text
    else:
        title = "No title"

    #get intro text.
    startText = None
    if soup.find("span",{"class":"ellipsis"}):
        span = True
        findSpanA = soup.find("span",{"class":"ellipsis"}).text
        findSpanB = soup.find("span",{"class":value}).text 
        startText = findSpanA + " " + findSpanB
    else:
        span = False
        startText = " "
        findArticle = soup.find("section",{"data-test-ui":"article__body"})
        findStart = findArticle.find_all("p",{"class":""})

    #text body
    findText = soup.find_all(["p","h2", "div"],{"class":value})

    display(HTML('<h1> %s </h1>' % title))
    display(HTML('<p>  </p>'))

    if span:
        display(HTML('<p> %s </p>' % startText))
    else:
        for i in findStart:
            display(HTML('<p> %s </p>' % i.text))

    display(HTML('<p>  </p>'))
    for i in findText:
        #sub-headings
        if i.name == "h2":
            display(HTML('<h2> %s </h2>' % i.text))
        # text
        elif i.name == "p":
            display(HTML('<p> %s </p>' % i.text))
            display(HTML('<p>  </p>'))

        ##images
        elif i.name == "div":
                #get 730px resolution image for display which is the 4th item in the array
                if i.find("img"):
                    images = i.find("img")["data-srcset"]
                    image = images.split(",")
                    imgUrl = image[3].split(" ")[0]
                    display(HTML('<img src="%s" alt="" width="500" height="600">' % imgUrl))
    

In [47]:
def NZHarold():
    url = input("Enter NZhearld URL: ")
    
    #soup it
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    #find hidden class
    classValue = findClass(soup)
    
    #if the article is free......
    if classValue == "na":
        display(HTML('<h1> This article is free silly! </h1>'))
    else:
        getContents(soup, classValue)


In [49]:
NZHarold()


Enter NZhearld URL: https://www.nzherald.co.nz/entertainment/jonathan-dean-talks-to-the-highest-grossing-actor-of-all-time-samuel-l-jackson/LGDH7V7WV6GFE3VPVQSG7TA3M4/
premiumm
